In [1]:
"""
코루틴
- yield : 생산한다/양보한다
- yield를 가진 함수. 그러나, yield 문이 표현식의 오른쪽에 옴/값을 생성하지 않는 경우도 있음
"""

'\n코루틴\n- yield : 생산한다/양보한다\n- yield를 가진 함수. 그러나, yield 문이 표현식의 오른쪽에 옴/값을 생성하지 않는 경우도 있음\n'

In [11]:
def simple_coroutine() : 
    print('-> coroutine started!')
    x = yield
    print('-> coroutine received : ', x)

my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f7c4963ac10>

In [12]:
next(my_coro)

-> coroutine started!


In [13]:
my_coro.send(42)

-> coroutine received :  42


StopIteration: 

In [15]:
from inspect import getgeneratorstate

def simple_coro2(a) :
    print('-> Started : a = ', a)
    b = yield a
    print('-> Received : b = ', b)
    c = yield a + b
    print('-> Received : c = ', c)


my_coro2 = simple_coro2(14)
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [16]:
next(my_coro2)

-> Started : a =  14


14

In [17]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [18]:
my_coro2.send(28)

-> Received : b =  28


42

In [19]:
my_coro2.send(99)

-> Received : c =  99


StopIteration: 

In [20]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

In [35]:
def averager() : 
    """코루틴을 이용해서 이동 평균을 구하는 방법"""
    total = 0.0
    count = 0
    average = None
    
    
    while True :
        term = yield average
        total += term
        count += 1
        average = total/count
        

In [36]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [37]:
coro_avg.send(30)

20.0

In [38]:
coro_avg.send(5)

15.0

In [39]:
from functools import wraps

def coroutine(func) : 
    """데커레이터 : 'func'를 기동해서 첫 번째 'yield'까지 진행함"""
    @wraps(func)
    def primer(*args, **kwargs) : 
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    
    return primer



In [54]:
# from inspect import getgeneratorstate
# from coroutil import coroutine

@coroutine
def averager() :
    total = 0.0
    count = 0
    average = None
    
    while True : 
        term = yield average
        total += term
        count += 1
        average = total/count
    

coro_avg = averager()
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [55]:
coro_avg.send(40)

40.0

In [56]:
coro_avg.send(50)

45.0

In [57]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [58]:
coro_avg.send(60)

StopIteration: 

In [70]:
"""
코루틴의 예외 처리 방법을 설명하기 위한 제너레이터
"""

class DemoException(Exception) : 
    """설명에 사용할 예외 유형"""
    
def demo_exc_handling() : 
    print('-> coroutine started!!')
    
    while True :
        try : 
            x = yield
        except DemoException : 
            print('*** DemoException handled. Continuing...')
        else : 
            print('-> coroutine received : {!r}'.format(x))
            
    raise RuntimeError('This line should never run!!')
    
def demo_finally() : 
    print('-> coroutine started!!')
    try : 
        while True : 
            try : 
                x = yield
                
            except DemoException : 
                print('*** DemoException handled. Continuing...')
            else : 
                print('-> coroutine received : {!r}'.format(x))
                
    finally : 
        print('-> coroutine ending!!')

In [71]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started!!


In [72]:
exc_coro.send(11)

-> coroutine received : 11


In [73]:
exc_coro.send(22)

-> coroutine received : 22


In [74]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [75]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [76]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started!!


In [77]:
exc_coro.send(11)

-> coroutine received : 11


In [78]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [79]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [80]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager() : 
    total = 0.0
    count = 0
    average = None
    
    while True : 
        term = yield
        
        if term is None : 
            break
            
        total += term
        count += 1
        average = total / count
    
    return Result(count, average)



In [90]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)


TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [91]:
try : 
    coro_avg.send(None)
    
except StopIteration as exc : 
    result = exc.value
    
print(result)

None


In [95]:
"""
yield from 사용

- 새로운 언어 구성체
- for 루프 안의 yield에 대한 단축문으로 사용할 수 있음
- 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어줌. 즉, 이 둘이 값을
  직접 주고받으며, 중간에 있는 코루틴이 판에 박힌 듯한 예외 처리 코드를 구현할 필요 없이 예외를 직접 던질
  수 있음 
- 코루틴 위임 할 수 있음 

"""

def gen() : 
    for c in 'AB' : 
        yield c
        
    for i in range(1, 3) : 
        yield i

list(gen())

['A', 'B', 1, 2]

In [96]:
def gen() : 
    yield from 'AB'
    yield from range(1, 3)
    
list(gen())

['A', 'B', 1, 2]

In [97]:
def chain(*iterables) : 
    for it in iterables : 
        yield from it
        
s = 'ABC'
t = tuple(range(3))

list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [99]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager() : 
    total = 0.0
    count = 0
    average = None
    
    while True : 
        term = yield
        
        if term is None : 
            break
            
        total += term
        count += 1
        average = total/count
        
    return Result(count, average)

# 대표 제너레이터
def grouper(results, key) : 
    while True : 
        results[key] = yield from averager()
        

# 호출자
def main(data) : 
    results = {}
    
    for key, values in data.items() : 
        group = grouper(results, key)
        next(group)
        
        for value in values : 
            group.send(value)
            
        group.send(None)
        
    report(results)


# 실행 결과 보고서 
def report(results) : 
    for key, result in sorted(results.items()) : 
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, 
                                                    result.average, unit))
        
data = {
    'girls;kg' : 
        [40.9, 38.5, 44.3, 42.2, 45.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m' :
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43], 
    'boys;kg' : 
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m' : 
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
}

if __name__ == '__main__' : 
    main(data)


 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
11 girls averaging 42.33kg
10 girls averaging 1.43m


In [102]:
RESULT = yield from EXPR
_i = iter(EXPR)

try : 
    _y = next(_i)
    
except StopIteration as _e : 
    _r = _e.value
else : 
    while 1 : 
        _s = yield _y
        
        try : 
            _y = _i.send(_s)
        except StopIteration as _e : 
            _r = _e.value
            break

RESULT = _r


SyntaxError: 'yield' outside function (668436811.py, line 1)

In [107]:
import EXPR
RESULT = yield from EXPR

_i = iter(EXPR)

try : 
    _y = next(_i)
except StopIteration as _e : 
    _r = _e.value
else : 
    while 1 : 
        try : 
            _s = yield _y
            
        except GeneratorExit as _e : 
            try : 
                _m = _i.close 
            except AttributeError : 
                pass
            else : 
                _m()
            raise _e
        except BaseException as _e : 
            _x = sys.exc_info()
            
            try : 
                _m = _i.throw
            except AttributeError : 
                raise _e
            else : 
                try : 
                    _y = _m(*_x)
                except StopIteration as _e : 
                    _r = _e.value
                    break
        else : 
            try : 
                if _s is None : 
                    _y = next(_i)
                else :
                    _y = _i.send(_s)
                    
            except StopIteration as _e : 
                _r = _e.value
                break
                
    RESULT = _r

ModuleNotFoundError: No module named 'EXPR'

In [1]:
class Simulator : 
    def __init__(self, procs_map) : 
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time) : 
        """시간이 끝날 때까지 이벤트를 스케줄링하고 출력함"""
        # 각 택시의 첫 번째 이벤트를 스케줄링함
        for _, proc in sorted(self.procs.items()) : 
            first_event = next(proc)
            self.events.put(first_event)
            
        # 시뮬레이션 핵심 루프
        sim_time = 0
        while sim_time < end_time : 
            if self.events.empty() : 
                print('*** end of events ***')
                break
                
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event 
            print('taxi : ', proc_id, proc_id * '  ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            
            try : 
                next_event = active_proc.send(next_time)
            except StopIteration : 
                del self.procs[proc_id]
            else : 
                self.events.put(next_event)
        else : 
            msg = '*** end of simulation time : {} events pending ***'
            print(msg.format(self.events.qsize()))
    